Single-cell: Dynamic clarifications -> researcher -> guided answer (Groq streaming + Tavily)
equirements: groq, langchain_tavily, python-dotenv. .env must contain GROQ_API_KEY and TAVILY_API_KEY.

In [4]:

import os, json, re
from typing import List, Dict, Any, Iterator
from dotenv import load_dotenv

load_dotenv()
API_KEY_GROQ = os.getenv("GROQ_API_KEY")
API_KEY_TAVILY = os.getenv("TAVILY_API_KEY")
if not API_KEY_GROQ or not API_KEY_TAVILY:
    raise RuntimeError("Please set GROQ_API_KEY and TAVILY_API_KEY in your .env")

from groq import Groq
from langchain_tavily import TavilySearch

# clients
groq_client = Groq(api_key=API_KEY_GROQ)
tavily_tool = TavilySearch(max_results=6)

# ---- Templates ----
PROMPT_GENERATE_CLARIFICATIONS = """
You are a meta assistant. The user requested: "{user_input}"
Known context (json): {clarified_info_json}
Return a JSON array (only) of up to {max_q} targeted clarifying question objects.
Each object should be: {{"q":"The question text","field":"field_key"}} where field_key is a short hint (dates, days, budget, region, language, scope, other).
If no clarification is needed, return an empty array [].
Do not include commentary.
"""

PROMPT_FINAL_ANSWER = """
You are an expert assistant. Use:
- User request: {user_input}
- Clarified info (json): {clarified_info}
- Research snippets (short json array): {research_snippets}

Produce a clear, exact, user-facing answer and actionable guidance (no raw URLs). Include:
1) A concise summary of the result (one or two lines).
2) If relevant, step-by-step guidance or day-by-day plan.
3) Concrete numbers where available (prices, durations, counts).
4) Practical next steps the user can take (book flights, pack list, commands, queries).
If research_snippets is empty, say you couldn't find direct references but provide a best-effort answer with guidance.
Stream the answer in natural chunks.
"""

# ---- Helpers ----
def safe_chunk_text(choice) -> str:
    try:
        if hasattr(choice, "delta") and getattr(choice.delta, "content", None):
            return choice.delta.content or ""
        if hasattr(choice, "message") and getattr(choice.message, "content", None):
            return choice.message.content or ""
    except Exception:
        pass
    return ""

def call_groq(messages: List[Dict[str,str]], stream: bool=False, max_tokens: int=300, temperature: float=0.0):
    return groq_client.chat.completions.create(messages=messages, model="llama-3.3-70b-versatile",
                                               stream=stream, max_completion_tokens=max_tokens, temperature=temperature)

def ask_for_clarifications(user_input: str, clarified_info: Dict[str,Any], max_q: int=2) -> List[Dict[str,str]]:
    prompt = PROMPT_GENERATE_CLARIFICATIONS.format(
        user_input=user_input,
        clarified_info_json=json.dumps(clarified_info, ensure_ascii=False),
        max_q=max_q
    )
    messages = [
        {"role":"system","content":"You output JSON-only: an array of question objects."},
        {"role":"user","content":prompt}
    ]
    resp = call_groq(messages, stream=False, max_tokens=200, temperature=0.0)
    try:
        text = resp.choices[0].message.content.strip()
    except Exception:
        text = str(resp)
    # parse JSON - robust
    questions = []
    try:
        arr = json.loads(text)
        if isinstance(arr, list):
            for item in arr[:max_q]:
                if isinstance(item, dict) and "q" in item:
                    questions.append({"q": item["q"].strip(), "field": item.get("field")})
    except Exception:
        # fallback: try to extract lines that look like questions
        for line in text.splitlines():
            line = line.strip()
            if line.endswith("?"):
                questions.append({"q": line, "field": None})
            if len(questions) >= max_q:
                break
    return questions[:max_q]

def map_answer(field_hint: str, question: str, answer: str, ctx: Dict[str,Any]):
    a = answer.strip()
    if not a:
        return
    lowq = (question or "").lower()
    if field_hint in ("dates","date") or any(w in lowq for w in ["date","when","travel","departure","return"]):
        ctx["dates"] = a
    elif field_hint in ("days","duration") or any(w in lowq for w in ["day","days","duration","length"]):
        m = re.search(r"\b(\d{1,2})\b", a)
        ctx["days"] = int(m.group(1)) if m else a
    elif field_hint in ("budget","price") or "budget" in lowq:
        ctx["budget"] = a
    elif field_hint in ("region","city","location") or any(w in lowq for w in ["city","where","region","country","to","from"]):
        ctx["region"] = a
    elif field_hint in ("language","lang"):
        ctx["language"] = a
    else:
        # fallback storing
        key = f"misc_{len(ctx)+1}"
        ctx[key] = a

def run_research(user_input: str, clarified_info: Dict[str,Any]) -> List[Dict[str,Any]]:
    # build query: combine user_input + clarified info keys
    parts = [user_input]
    for k,v in clarified_info.items():
        parts.append(f"{k}:{v}")
    query = " ".join(str(p) for p in parts if p)
    try:
        res = tavily_tool.invoke({"query": query})
        results = res.get("results", []) if isinstance(res, dict) else []
    except Exception as e:
        print("Research tool error:", e)
        results = []
    return results

def stream_final_answer(user_input: str, clarified_info: Dict[str,Any], research_results: List[Dict[str,Any]]) -> Iterator[str]:
    # prepare small snippets array
    snippets = []
    for r in (research_results or [])[:6]:
        s = (r.get("snippet") or r.get("content") or "").replace("\n"," ")
        snippets.append((r.get("title") or r.get("url") or "") + " — " + (s[:300] + ("..." if len(s)>300 else "")))
    messages = [
        {"role":"system","content":"You are a precise assistant. Provide exact facts and practical guidance (no raw links)."},
        {"role":"user","content": PROMPT_FINAL_ANSWER.format(
            user_input=user_input,
            clarified_info=json.dumps(clarified_info, ensure_ascii=False),
            research_snippets=json.dumps(snippets, ensure_ascii=False)
        )}
    ]
    # try streaming; on failure fallback to one-off generation
    try:
        stream = call_groq(messages, stream=True, max_tokens=700, temperature=0.4)
    except Exception:
        try:
            resp = call_groq(messages, stream=False, max_tokens=700, temperature=0.4)
            yield resp.choices[0].message.content
            return
        except Exception as e:
            yield f"(Failed to generate answer: {e})"
            return

    got = False
    for chunk in stream:
        try:
            txt = safe_chunk_text(chunk.choices[0])
            if txt:
                got = True
                yield txt
        except Exception:
            continue
    if not got:
        # fallback single call
        try:
            resp = call_groq(messages, stream=False, max_tokens=700, temperature=0.4)
            try:
                yield resp.choices[0].message.content
            except Exception:
                yield str(resp)
        except Exception as e:
            yield f"(Fallback failed: {e})"

# ---- Interactive one-shot flow (asks up to 2 clarifying Qs, then research, then final answer) ----
def dynamic_qna_once():
    user_input = input("You: ").strip()
    if not user_input:
        print("Please type a request.")
        return

    clarified_info: Dict[str,Any] = {}
    # 1) ask up to 2 clarifying questions based on prompt
    questions = ask_for_clarifications(user_input, clarified_info, max_q=2)
    if questions:
        print("Bot: I have a couple quick questions to make the answer exact.")
    for qobj in questions:
        qtext = qobj.get("q") or ""
        field = qobj.get("field")
        if not qtext:
            continue
        print("Bot (clarify):", qtext)
        ans = input("You (answer): ").strip()
        if ans.lower() in ("skip","n/a","none","no"):
            continue
        map_answer(field, qtext, ans, clarified_info)

    # 2) research using clarified_info
    print("Bot: researching the information now (I will summarize findings)...")
    research_results = run_research(user_input, clarified_info)

    # 3) produce final exact guidance (streamed)
    print("\nBot (answer):")
    for chunk in stream_final_answer(user_input, clarified_info, research_results):
        print(chunk, end="", flush=True)
    print("\n\n--- End ---")
    # optionally show small trace (counts)
    print(f"\n(Research results used: {len(research_results)})")

# Run once
dynamic_qna_once()


Bot: I have a couple quick questions to make the answer exact.
Bot (clarify): What are your travel dates or how many days do you have for the trip from Hyderabad to New York?
Bot (clarify): What is your approximate budget for the trip?
Bot: researching the information now (I will summarize findings)...

Bot (answer):
**Trip Summary**
To travel from Hyderabad to New York by the cheapest transport, the most affordable round-trip flights start at $592, with one-way flights available from $362. 

**Step-by-Step Guidance**
Given your budget of $50,000 and a 5-day trip from October 20, 2025, here's a suggested plan:
1. **Day 1-2: Travel to New York**
   - Book a round-trip flight from Hyderabad to New York. The cheapest option found is $592, which fits within your budget.
   - Consider flying with airlines like IndiGo or Turkish Airlines, which offer affordable rates with layovers.

2. **Day 3-5: Explore New York**
   - After arriving in New York, spend your days exploring the city. Given yo